## Ideas for visualisation :
* maps of users
    * scaled with nb followers 
    * scaled with nb stars
* commit / user heatmap
* dominant languages per canton
* Cooperation graph (find which unis have common projects)
* map of repos